In [ ]:
import pandas as pd
import psycopg2

# Configuration PostgreSQL
DB_HOST = 'localhost'
DB_PORT = 5432
DB_NAME = 'Base1'
DB_USER = 'waouhmonde'
DB_PASSWORD = 'waouhmonde'

# Liste des secteurs à extraire
secteurs = [
    {"nom": "GONDREVILLE l'orcher", "code_postal": "76"},
    {"nom": "ROQUEBRUNE sur argent", "code_postal": "83"},
    {"nom": "AURILLAC", "code_postal": "15"},
    {"nom": "PRIVAS", "code_postal": "07"},
    {"nom": "VALENCE", "code_postal": "26"},
    {"nom": "CRÈCHE SUR SOANE", "code_postal": "71"}
]

# Dictionnaire pour stocker les DataFrames de chaque secteur
result_dfs = {}

# Boucle pour extraire les données de chaque secteur
for secteur in secteurs:
    ville_cible = secteur["nom"]
    code_postal_cible = secteur["code_postal"]

    # Définissez la requête SQL pour récupérer les lignes correspondant au secteur actuel
    query = "SELECT * FROM wmbase1 WHERE ville = %s AND code_postal LIKE %s"

    # Établissez une connexion à la base de données PostgreSQL
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )

    # Exécutez la requête SQL et créez un DataFrame pour le secteur actuel
    df = pd.read_sql_query(query, conn, params=[ville_cible, f'{code_postal_cible}%'])

    # Fermez la connexion à la base de données
    conn.close()

    # Stockez le DataFrame du secteur dans le dictionnaire en utilisant le nom du secteur comme clé
    result_dfs[secteur["nom"]] = df

# Enregistrez chaque DataFrame dans un fichier CSV et un classeur Excel
for nom_secteur, df_secteur in result_dfs.items():
    # Enregistrez dans un fichier CSV
    csv_filename = f"{nom_secteur.replace(' ', '_')}.csv"
    df_secteur.to_csv(csv_filename, index=False)

    # Enregistrez dans un fichier Excel (XLSX)
    excel_filename = f"{nom_secteur.replace(' ', '_')}.xlsx"
    df_secteur.to_excel(excel_filename, index=False)

    print(f"Données pour le secteur {nom_secteur} enregistrées dans {csv_filename} et {excel_filename}")

